In [193]:
#importing
import numpy as np
import pandas as pd

data_wd = "C:/Users/Lenovo/Desktop/git/STAT4011_Project1/data/Lianjia_raw.csv"
df1 = pd.read_csv(data_wd,header=None).astype("str")

dropped_column = [2,3,5,11,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,48,50,52,54,56,58,59,60,62,63,64,65,66,67,69]
df = df1.drop(dropped_column,axis=1)
df.index = range(21299)
df.columns = ["city","district","price","layout","rel_floor","total_floor","area_size","config","building_type","direction","structure","renovation_status","no_of_units","elevator","date_on_market","transact_property","last_transact_date","function","age_limit","property_belong","upload_photo","loc","avail_visit_time"]

In [194]:
#converting price to float
pr = df["price"]
#for i in range(len(pr)):
#  if(pr[i].find("万")==-1):
#    print(i)
#df.iloc[2121],改一下那条record
r1_reformat = [ "shanghai","pudong","371万","2室2厅1厨1卫","中楼层","共6层","72.88㎡","平层","板楼","南 北","砖混结构","简装","一梯两户","无","2022-07-09","商品房","2006-06-06","普通住宅","满五年","共有","已上传房本照片","\xa0外环外","\n" ]
r2_reformat = ["shanghai","pudong","213万","2室1厅1厨1卫","中楼层","共6层","70.31㎡","平层","板楼","南","砖混结构","简装","一梯四户","无","2022-03-03","商品房","2020-03-23","普通住宅","满两年","非共有","已上传房本照片","\n","\n"]
df.iloc[2121]=r1_reformat
df.iloc[2165]=r2_reformat
for i in range(len(pr)):
  price = df.iloc[i]["price"]
  price = price.strip()
  price = price[:-1]
  price = float(price)*10000
  df.loc[i,"price"]=price

In [195]:
# area size to float
asi = df["area_size"]
for i in range(len(asi)):
  size = df.iloc[i]["area_size"]
  size = size.strip()
  size = size.strip("'")
  size = size[:-1]
  size = float(size)
  df.loc[i,"area_size"]=size

In [196]:
#layout_conversion
lo = df["layout"]
for i in range(len(lo)):
  layout = df.iloc[i]["layout"]
  layout = layout.strip()
  layout = layout.strip('"')
  layout = layout.strip("[")
  layout = layout.strip("'")
  df.loc[i,"layout"]=layout

In [197]:
for i in range(len(lo)):
  layout = df.iloc[i]["layout"]
  if(len(layout)!=8):
    print(i)
#室厅厨卫 4*n matrices, item 13972 as an exception

13972


In [198]:
#split the column into 4 columns
lo = df["layout"]
bedroom_no = []
parlour_no = []
kitchen_no = []
toilet_no = []
for i in range(len(lo)):
  strr = lo[i]
  if i == 13972:
    bedroom_no.append(10)
    parlour_no.append(1)
    kitchen_no.append(0)
    toilet_no.append(1)
  #elif strr.find("室")!=1 or strr.find("厅")!=3 or strr.find("厨")!=5 or strr.find("卫")!=7:
  else:
    bedroom_no.append(int(strr[0]))
    parlour_no.append(int(strr[2]))
    kitchen_no.append(int(strr[4]))
    toilet_no.append(int(strr[6]))
df['bedrm_no']=bedroom_no
df['parlour_no']=parlour_no
df['kitchen_no']=kitchen_no
df['toilet_no']=toilet_no

In [199]:
#strip formatting rel floor
rf = df["rel_floor"]
for i in range(len(rf)):
  relf = df.iloc[i]["rel_floor"]
  relf = relf.strip()
  relf = relf.strip("'")
  df.loc[i,"rel_floor"]= relf

In [200]:
#total_floor
tf = df["total_floor"]
for i in range(len(tf)):
  totf = df.iloc[i]["total_floor"]
  totf = totf.strip()
  totf = totf.strip("'")
  totf = totf.strip(")")
  totf = totf[:-1]
  totf = int(totf[1:])
  df.loc[i,"total_floor"] = totf

In [201]:
#strip formatting config
conf = df["config"]
for i in range(len(conf)):
  cf = df.iloc[i]["config"]
  cf = cf.strip()
  cf = cf.strip("'")
  df.loc[i,"config"]= cf

In [202]:
#strip formatting building_type
building_type = df["building_type"]
for j in range(len(building_type)):
  bt = df.iloc[j]["building_type"]
  bt = bt.strip()
  bt = bt.strip("'")
  df.loc[j,"building_type"] = bt

# Direction "strip" is pre-processed here

In [203]:
#strip formatting direction
direction = df["direction"]
for j in range(len(direction)):
  dr = df.iloc[j]["direction"]
  dr = dr.strip()
  dr = dr.strip("'")
  df.loc[j,"direction"] = dr

# Structure "Strip" is pre-processed here

In [204]:
#strip formatting 
easy_strip = ["structure","renovation_status","no_of_units","elevator","date_on_market","transact_property","last_transact_date","function","age_limit","property_belong","upload_photo","loc","avail_visit_time"]
structure = df["structure"]
for i in easy_strip:
    for j in range(len(structure)):
      bti = df.iloc[j][i]
      bti = bti.strip()
      bti = bti.strip("'")
      df.loc[j,i] = bti

In [205]:
#available time data
tim = df["avail_visit_time"]
for j in range(len(tim)):
  tim = df.iloc[j]["avail_visit_time"]
  tim = tim.strip("]")
  tim = tim.strip("'")
  df.loc[j,"avail_visit_time"]=tim
#upload photo
#location data
pho = df["upload_photo"]
for j in range(len(pho)):
  pho = df.iloc[j]["upload_photo"]
  pho = pho.strip("]")
  pho = pho.strip("'")
  df.loc[j,"upload_photo"]=pho

In [206]:
df.head()

,city,district,price,layout,rel_floor,total_floor,area_size,config,building_type,direction,...,function,age_limit,property_belong,upload_photo,loc,avail_visit_time,bedrm_no,parlour_no,kitchen_no,toilet_no
0,shanghai,pudong,4350000.0,2室2厅1厨1卫,中楼层,6,52.01,平层,板楼,南 西,...,普通住宅,满五年,共有,已上传房本照片,\xa0内环内'],提前预约随时可看,2,2,1,1
1,shanghai,pudong,2750000.0,1室1厅1厨1卫,高楼层,6,49.77,平层,板楼,南,...,普通住宅,满五年,共有,已上传房本照片,\xa0外环外'],有租户需预约,1,1,1,1
2,shanghai,pudong,3050000.0,2室1厅1厨1卫,高楼层,5,65.75,平层,板楼,南,...,普通住宅,满五年,共有,已上传房本照片,\xa0外环外'],提前预约随时可看,2,1,1,1
3,shanghai,pudong,4500000.0,2室2厅1厨1卫,高楼层,6,86.37,平层,板楼,南,...,普通住宅,满五年,共有,已上传房本照片,\xa0外环外'],提前预约随时可看,2,2,1,1
4,shanghai,pudong,2230000.0,2室1厅1厨1卫,高楼层,21,71.22,平层,板楼,南,...,普通住宅,满两年,非共有,已上传房本照片,\xa0'],提前预约随时可看,2,1,1,1


In [207]:
#strip formatting building_type
building_type = df["building_type"]
for j in range(len(building_type)):
  bt = df.iloc[j]["building_type"]
  bt = bt.strip()
  bt = bt.strip("'")
  df.loc[j,"building_type"] = bt

In [208]:
date1 = df["date_on_market"]
date1.tail()

21294    2022-08-03
21295    2019-10-17
21296    2020-12-28
21297    2021-02-28
21298    2022-07-17
Name: date_on_market, dtype: object

In [209]:
import datetime as dt
now = dt.date.today()
date1 = df["date_on_market"]
date2 = df["last_transact_date"]
for i in range(len(date1)):
 datetemp1 = now -dt.date.fromisoformat(str(date1[i]))
 df.loc[i,"date_on_market"]=datetemp1

In [210]:
for i in range(len(date2)):
  if date2[i].find("暂无数据")!=-1:
    continue
  else:
    datetemp2 = now-dt.date.fromisoformat(date2[i])
    df.loc[i,"last_transact_date"]=datetemp2

# "loc" is again stripped here, at the end of the following cell

In [212]:
column_name = ["city","district","rel_floor","config","building_type","direction","structure","renovation_status","no_of_units","elevator","date_on_market","transact_property","last_transact_date","function","age_limit","property_belong","upload_photo","loc","avail_visit_time"]
for j in column_name:
  for i in range(len(date1)):
    if(type(df[j][i])!=str):
      continue
    elif(df[j][i].find("暂无数据")!=-1):
      df.loc[i,j]=np.NaN
    elif(df[j][i].find("\n")!=-1):
      df.loc[i,j]=np.NaN
for t in range(len(date1)):
  str1 = df["loc"][t]
  if(type(str1)==str):
    str1 = str1.strip('"')
    str1 = str1.strip("]")
    str1 = str1.strip("'")
    df.loc[t,"loc"] = str1

In [213]:
df["date_on_market"] # now a datetime.timedelta object, maybe transformed to integer(no.of days later on)

0          36 days, 0:00:00
1          20 days, 0:00:00
2         178 days, 0:00:00
3         193 days, 0:00:00
4          62 days, 0:00:00
                ...        
21294      59 days, 0:00:00
21295    1080 days, 0:00:00
21296     642 days, 0:00:00
21297     580 days, 0:00:00
21298      76 days, 0:00:00
Name: date_on_market, Length: 21299, dtype: object

In [215]:
#elevator
ele = df["elevator"]
for i in range(len(ele)):
  if(ele[i]=="有"):
    df.loc[i,"elevator"]=1
  elif ele[i]=="无":
    df.loc[i,"elevator"]=0

to be processed:\
*no_of_units* column. concatenate the string into two parts, output all possible values in chinese character. substitue them with decimal number representation. compute the proportion. substitute the previous column with new arrays.

In [216]:
ar1 = df["no_of_units"]
stair = []
unit = []
for i in range(len(ar1)):
  str1 = df["no_of_units"][i]
  if (str1.find("梯")== -1):
    stair.append(np.NaN)
    unit.append(np.NaN)
  else:
    inde = str1.index("梯")
    if inde > 1:
      stair.append(str1[0:(inde-1)])
    else:
      stair.append(str1[0])
    str1 = str1[:-1]
    unit.append(str1[(inde+1):])

x = np.array(stair)
y = np.array(unit)
xu = np.unique(x)
yu = np.unique(y)
xu1 = xu.tolist()
yu1 = yu.tolist()

In [217]:
print(xu)
print(yu)
xup = [np.NaN,1,7,3,2,9,2,5,8,6,10,4]
yup = [np.NaN,1,100,7,77,74,3,30,31,37,33,32,35,36,34,2,9,20,21,27,23,29,22,25,28,26,24,5,50,51,57,53,59,52,55,58,56,8,80,84,6,65,66,10,11,17,13,19,12,15,18,16,14,4,40,41,47,43,42,45,46]


['nan' '一' '七' '三' '两' '九' '二' '五' '八' '六' '十' '四']
['nan' '一' '一百' '七' '七十七' '七十四' '三' '三十' '三十一' '三十七' '三十三' '三十二' '三十五'
 '三十六' '三十四' '两' '九' '二十' '二十一' '二十七' '二十三' '二十九' '二十二' '二十五' '二十八' '二十六'
 '二十四' '五' '五十' '五十一' '五十七' '五十三' '五十九' '五十二' '五十五' '五十八' '五十六' '八' '八十'
 '八十四' '六' '六十五' '六十六' '十' '十一' '十七' '十三' '十九' '十二' '十五' '十八' '十六' '十四'
 '四' '四十' '四十一' '四十七' '四十三' '四十二' '四十五' '四十六']


In [218]:
xu1 = xu.tolist()
yu1 = yu.tolist()
for i in range(len(stair)):
  if(stair[i] in [np.nan]):
    continue
  else:
    stair[i]=xup[xu1.index(stair[i])]
for j in range(len(unit)):
  if(unit[j] in [np.nan]):
    continue
  else:
    unit[j]=yup[yu1.index(unit[j])]


In [219]:
df["stair"]=stair
df["unit"]=unit
st = np.array(stair)
un = np.array(unit)
proportion = np.divide(st,un)
print(len(proportion))
df["proportion"]=proportion

21299


C:\Users\Lenovo\AppData\Local\Temp/ipykernel_23440/3649857014.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df["stair"]=stair
C:\Users\Lenovo\AppData\Local\Temp/ipykernel_23440/3649857014.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df["unit"]=unit
C:\Users\Lenovo\AppData\Local\Temp/ipykernel_23440/3649857014.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(ax

In [220]:
rf = df["rel_floor"]
rfp = np.array(rf)
print(np.unique(rfp))

['中楼层' '低楼层' '高楼层']


In [221]:
middle_floor = []
lower_floor = []
high_floor = []
for i in range(len(rf)):
  if(rf[i]=="中楼层"):
    middle_floor.append(1)
    lower_floor.append(0)
    high_floor.append(0)
  elif rf[i]=="低楼层":
    middle_floor.append(0)
    lower_floor.append(1)
    high_floor.append(0)
  else:
    middle_floor.append(0)
    lower_floor.append(0)
    high_floor.append(1)
df["middle_floor"]=middle_floor
df["lower_floor"]=lower_floor
df["high_floor"]=high_floor


C:\Users\Lenovo\AppData\Local\Temp/ipykernel_23440/2030144815.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df["middle_floor"]=middle_floor
C:\Users\Lenovo\AppData\Local\Temp/ipykernel_23440/2030144815.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df["lower_floor"]=lower_floor
C:\Users\Lenovo\AppData\Local\Temp/ipykernel_23440/2030144815.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all col

In [222]:
config = df["config"]
con = np.array(config)
for i in range(len(con)):
  con[i]=str(con[i])
print(np.unique(con))

['108.88㎡' '182.54㎡' '303.19㎡' '356.3㎡' '403.3㎡' '538.45㎡' '555.51㎡' 'nan'
 '复式' '平层' '跃层' '错层']


In [223]:
for i in range(len(con)):
  if(con[i].find("㎡")!=-1):
    df.loc[i,"config"]==np.nan

In [224]:
duplex_config = []
levelling_config = []
duplex_yconfig = []
splitlevel_config = []
for i in range(len(con)):
  if con[i]=="复式":
    duplex_config.append(1)
    levelling_config.append(0)
    duplex_yconfig.append(0)
    splitlevel_config.append(0)
  elif con[i]=="平层":
    duplex_config.append(0)
    levelling_config.append(1)
    duplex_yconfig.append(0)
    splitlevel_config.append(0)
  elif con[i]=="错层":
    duplex_config.append(0)
    levelling_config.append(0)
    duplex_yconfig.append(0)
    splitlevel_config.append(1)
  elif con[i]=="跃层":
    duplex_config.append(0)
    levelling_config.append(0)
    duplex_yconfig.append(1)
    splitlevel_config.append(0)
  else:
    duplex_config.append(0)
    levelling_config.append(0)
    duplex_yconfig.append(0)
    splitlevel_config.append(0)
df["fushi"]=duplex_config
df["pingceng"]=levelling_config
df["cuoceng"]=splitlevel_config
df["yueceng"]=duplex_yconfig

C:\Users\Lenovo\AppData\Local\Temp/ipykernel_23440/1182751425.py:31: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df["fushi"]=duplex_config
C:\Users\Lenovo\AppData\Local\Temp/ipykernel_23440/1182751425.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df["pingceng"]=levelling_config
C:\Users\Lenovo\AppData\Local\Temp/ipykernel_23440/1182751425.py:33: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns

In [225]:
df["building_type"]

0          板楼
1          板楼
2          板楼
3          板楼
4          板楼
         ... 
21294      塔楼
21295    板塔结合
21296    板塔结合
21297      塔楼
21298     NaN
Name: building_type, Length: 21299, dtype: object

In [226]:
bt = df["building_type"]
btt = np.array(bt)
for i in range(len(btt)):
  btt[i]=str(btt[i])
print(np.unique(btt))

['nan' '塔楼' '平房' '未知结构' '板塔结合' '板楼' '框架结构' '砖混结构' '钢混结构']


In [227]:
anomaly_list = []
for i in range(len(btt)):
  if(btt[i]=="钢混结构" or btt[i]=="未知结构" or btt[i]=="框架结构" or btt[i]=="砖混结构"):
    anomaly_list.append(i)
anomaly_list

[804,
 807,
 1166,
 1169,
 1183,
 1243,
 4174,
 5398,
 5479,
 5604,
 5651,
 5666,
 5730,
 6602,
 10736,
 11783,
 12230,
 14051,
 14060,
 14158,
 14193,
 14236,
 14241,
 14278,
 14286,
 14365,
 14367,
 14371,
 14372,
 14376,
 14378,
 14382,
 14383,
 14384,
 14385,
 14387,
 14388,
 14391,
 14394,
 14395,
 14594,
 14604,
 14799,
 14987,
 14988,
 15107,
 15154,
 15295,
 15372,
 15872,
 17057,
 17105,
 17109,
 17115,
 17116,
 17135,
 17150,
 17327,
 17354,
 17500,
 18529,
 18717,
 18846,
 19019,
 19111,
 19697,
 20693,
 20752,
 20777,
 20781,
 20784,
 20785,
 20786,
 20787,
 20791,
 20794,
 20796,
 20878,
 20890,
 20923,
 21008,
 21009,
 21244]

In [228]:
for i in anomaly_list:
  struct = df.iloc[i]["building_type"]
  bt = df.iloc[i]["structure"]
  renova = df.iloc[i]["direction"]
  df.col[i,"building_type"]=bt
  df.col[i,"structure"]=struct
  df.col[i,"renovation_status"]=renova
  df.col[i,"direction"]=df_correct.iloc[i+3,11]

AttributeError: 'DataFrame' object has no attribute 'col'

In [ ]:
#building type
bt = df["building_type"]
btt = np.array(bt)
for i in range(len(btt)):
  btt[i]=str(btt[i])
print(np.unique(btt))

In [ ]:
#building type dummy 
shuangpin = []
talou = []
pingfang = []
taban_combine = []
banlou = []
dudong = []
lianpai = []
for i in range(len(bt)):
  if bt[i]=="双拼":
    shuangpin.append(1)
    talou.append(0)
    pingfang.append(0)
    taban_combine.append(0)
    banlou.append(0)
    dudong.append(0)
    lianpai.append(0)
  elif bt[i]=="叠拼":
    shuangpin.append(0)
    talou.append(0)
    pingfang.append(0)
    taban_combine.append(0)
    banlou.append(0)
    dudong.append(0)
    lianpai.append(0)
  elif bt[i]=="塔楼":
    shuangpin.append(0)
    talou.append(1)
    pingfang.append(0)
    taban_combine.append(0)
    banlou.append(0)
    dudong.append(0)
    lianpai.append(0)
  elif bt[i]=="平房":
    shuangpin.append(0)
    talou.append(0)
    pingfang.append(1)
    taban_combine.append(0)
    banlou.append(0)
    dudong.append(0)
    lianpai.append(0)
  elif bt[i]=="板塔结合":
    shuangpin.append(0)
    talou.append(0)
    pingfang.append(0)
    taban_combine.append(1)
    banlou.append(0)
    dudong.append(0)
    lianpai.append(0)
  elif bt[i]=="板楼":
    shuangpin.append(0)
    talou.append(0)
    pingfang.append(0)
    taban_combine.append(0)
    banlou.append(1)
    dudong.append(0)
    lianpai.append(0)
  elif bt[i]=="独栋":
    shuangpin.append(0)
    talou.append(0)
    pingfang.append(0)
    taban_combine.append(0)
    banlou.append(0)
    dudong.append(1)
    lianpai.append(0)
  elif bt[i]=="联排":
    shuangpin.append(0)
    talou.append(0)
    pingfang.append(0)
    taban_combine.append(0)
    banlou.append(0)
    dudong.append(0)
    lianpai.append(1)
  else:
    shuangpin.append(np.nan)
    talou.append(np.nan)
    pingfang.append(np.nan)
    taban_combine.append(np.nan)
    banlou.append(np.nan)
    dudong.append(np.nan)
    lianpai.append(np.nan)

In [ ]:
df["shuangpin"]=shuangpin
df["talou"]=talou
df["pingfang"]=pingfang
df["taban_combine"]=taban_combine
df["banlou"]=banlou
df["dudong"]=dudong
df["lianpai"]=lianpai

In [ ]:
direct = df["direction"]
directt = np.array(direct)
for i in range(len(directt)):
  directt[i]=str(directt[i])
  t=directt[i]
  t = t.strip()
  t = t.strip('"')
  t = t.strip("'")
  directt[i]=t
print(np.unique(directt))


In [ ]:
east = []
southeast = []
south = []
southwest = []
west = []
northwest = []
north = []
northeast = []
for i in range(len(directt)):
  str1 = directt[i]
  arr1 = str1.split()
  for j in range(len(arr1)):
    if arr1[j]=="东":
      east.append(1)
    elif arr1[j]=="西":
      west.append(1)
    elif arr1[j]=="南":
      south.append(1)
    elif arr1[j]=="北":
      north.append(1)
    elif arr1[j]=="东南":
      southeast.append(1)
    elif arr1[j]=="西南":
      southwest.append(1)
    elif arr1[j]=="西北":
      northwest.append(1)
    elif arr1[j]=="东北":
      northeast.append(1)
  if len(east)==i:
    east.append(0)
  if len(west)==i:
    west.append(0)
  if len(north)==i:
    north.append(0)
  if len(south)==i:
    south.append(0)
  if len(northeast)==i:
    northeast.append(0)
  if len(southeast)==i:
    southeast.append(0)
  if len(southwest)==i:
    southwest.append(0)
  if len(northwest)==i:
    northwest.append(0)
df["south"]=south
df["west"]=west
df["north"]=north
df["east"]=east
df["northeast"]=northeast
df["southeast"]=southeast
df["northwest"]=northwest
df["southwest"]=southwest  

In [ ]:
struct = df["structure"]
struc = np.array(struct)
for i in range(len(struct)):
  struc[i] = str(struc[i])
print(np.unique(struc))

In [ ]:
unknown_struct = []
frame_struct = []
mixed_struct = []
brickwood_struct = []
brickmix_struct = []
steelmix_struct = []
steel_struct = []
for i in range(len(struc)):
  strr = struc[i]
  if strr == "未知结构":
    unknown_struct.append(1)
    frame_struct.append(0)
    mixed_struct.append(0)
    brickwood_struct.append(0)
    brickmix_struct.append(0)
    steelmix_struct.append(0)
    steel_struct.append(0)
  elif strr =="框架结构":
    unknown_struct.append(0)
    frame_struct.append(1)
    mixed_struct.append(0)
    brickwood_struct.append(0)
    brickmix_struct.append(0)
    steelmix_struct.append(0)
    steel_struct.append(0)
  elif strr =="混合结构":
    unknown_struct.append(0)
    frame_struct.append(0)
    mixed_struct.append(1)
    brickwood_struct.append(0)
    brickmix_struct.append(0)
    steelmix_struct.append(0)
    steel_struct.append(0)
  elif strr =="砖木结构":
    unknown_struct.append(0)
    frame_struct.append(0)
    mixed_struct.append(0)
    brickwood_struct.append(1)
    brickmix_struct.append(0)
    steelmix_struct.append(0)
    steel_struct.append(0)
  elif strr == "砖混结构":
    unknown_struct.append(0)
    frame_struct.append(0)
    mixed_struct.append(0)
    brickwood_struct.append(0)
    brickmix_struct.append(1)
    steelmix_struct.append(0)
    steel_struct.append(0)
  elif strr =="钢混结构":
    unknown_struct.append(0)
    frame_struct.append(0)
    mixed_struct.append(0)
    brickwood_struct.append(0)
    brickmix_struct.append(0)
    steelmix_struct.append(1)
    steel_struct.append(0)
  elif strr == "钢结构":
    unknown_struct.append(0)
    frame_struct.append(0)
    mixed_struct.append(0)
    brickwood_struct.append(0)
    brickmix_struct.append(0)
    steelmix_struct.append(0)
    steel_struct.append(1)
  else:
    unknown_struct.append(0)
    frame_struct.append(0)
    mixed_struct.append(0)
    brickwood_struct.append(0)
    brickmix_struct.append(0)
    steelmix_struct.append(0)
    steel_struct.append(0)
df["unknown_struct"]=unknown_struct
df["frame_struct"]=frame_struct
df["mixed_struct"]=mixed_struct
df["brickwood_struct"]=brickwood_struct
df["brickmix_struct"]=brickmix_struct
df["steelmix_struct"]=steelmix_struct
df["steel_struct"]=steel_struct 

In [ ]:
renov = df["renovation_status"]
reno = np.array(renov)
for i in range(len(reno)):
  reno[i]=str(reno[i])
print(np.unique(reno))

In [ ]:
other_reno = []
semi_reno = []
sim_reno = []
pro_reno = []
for i in range(len(reno)):
  ren = reno[i]
  if ren =="其他":
    other_reno.append(1)
    semi_reno.append(0)
    sim_reno.append(0)
    pro_reno.append(0)
  elif ren =="毛坯":
    other_reno.append(0)
    semi_reno.append(1)
    sim_reno.append(0)
    pro_reno.append(0)
  elif ren == "简装":
    other_reno.append(0)
    semi_reno.append(0)
    sim_reno.append(1)
    pro_reno.append(0)
  elif ren == "精装":
    other_reno.append(0)
    semi_reno.append(0)
    sim_reno.append(0)
    pro_reno.append(1)
  else:
    other_reno.append(0)
    semi_reno.append(0)
    sim_reno.append(0)
    pro_reno.append(0)
df["other_reno"]=other_reno
df["semi_reno"]=semi_reno
df["sim_reno"]=sim_reno
df["pro_reno"]=pro_reno

In [ ]:
# drop a dummy var for each categorical var, leave out direction
df = df.drop("lower_floor",axis=1)
df = df.drop("yueceng",axis=1)
df = df.drop("diepin",axis=1)
df = df.drop("steel_struct",axis=1)
df = df.drop("semi_reno",axis=1)

In [ ]:
df

In [ ]:
tp = df["transact_property"]
tpp = np.array(tp)
for i in range(len(tpp)):
  tpp[i]=str(tpp[i])
print(np.unique(tpp))

In [ ]:
resettle_housing = []
public_housing = []
for i in range(len(tp)):
  if tp[i]=="动迁安置房":
    resettle_housing.append(1)
    public_housing.append(0)
  elif tp[i]=="售后公房":
    resettle_housing.append(0)
    public_housing.append(1)
  else:
    resettle_housing.append(0)
    public_housing.append(0)
df["resettle_housing"]=resettle_housing
df["public_housing"]=public_housing


In [ ]:
func = df["function"]
funcc = np.array(func)
for i in range(len(funcc)):
  funcc[i]=str(funcc[i])
print(np.unique(funcc))

In [ ]:
villa_func = []
office_func = []
xinshi_func = []
jiushi_func = []
commonresi_func = []
oldapar_func = []
for i in range(len(funcc)):
  if funcc[i]=="老公寓":
    villa_func.append(0)
    office_func.append(0)
    xinshi_func.append(0)
    jiushi_func.append(0)
    commonresi_func.append(0)
    oldapar_func.append(1)
  elif func[i]=="别墅":
    villa_func.append(1)
    office_func.append(0)
    xinshi_func.append(0)
    jiushi_func.append(0)
    commonresi_func.append(0)
    oldapar_func.append(0)
  elif func[i]=="商业办公类":
    villa_func.append(0)
    office_func.append(1)
    xinshi_func.append(0)
    jiushi_func.append(0)
    commonresi_func.append(0)
    oldapar_func.append(0)
  elif func[i]=="新式里弄":
    villa_func.append(0)
    office_func.append(0)
    xinshi_func.append(1)
    jiushi_func.append(0)
    commonresi_func.append(0)
    oldapar_func.append(0)
  elif func[i]=="旧式里弄":
    villa_func.append(0)
    office_func.append(0)
    xinshi_func.append(0)
    jiushi_func.append(1)
    commonresi_func.append(0)
    oldapar_func.append(0)
  elif func[i]=="普通住宅":
    villa_func.append(0)
    office_func.append(0)
    xinshi_func.append(0)
    jiushi_func.append(0)
    commonresi_func.append(1)
    oldapar_func.append(0)
  elif func[i]=="花园洋房":
    villa_func.append(0)
    office_func.append(0)
    xinshi_func.append(0)
    jiushi_func.append(0)
    commonresi_func.append(0)
    oldapar_func.append(0)
  else:
    villa_func.append(np.nan)
    office_func.append(np.nan)
    xinshi_func.append(np.nan)
    jiushi_func.append(np.nan)
    commonresi_func.append(np.nan)
    oldapar_func.append(np.nan)
df["villa_func"]=villa_func
df["office_func"]=office_func
df["xinshi_func"]=xinshi_func
df["jiushi_func"]=jiushi_func
df["commonresi_func"]=commonresi_func
df["oldapar_func"]=oldapar_func

In [229]:
agel = df["age_limit"]
agell = np.array(agel)
for i in range(len(agell)):
  agell[i]=str(agell[i])
print(np.unique(agell))

['nan' '未满两年' '满两年' '满五年']


In [230]:
below_twoyr = []
beyond_twoyr = []
for i in range(len(agell)):
  if agell[i]=="未满两年":
    below_twoyr.append(1)
    beyond_twoyr.append(0)
  elif agell[i]=="满两年":
    beyond_twoyr.append(1)
    below_twoyr.append(0)
  elif agell[i]=="满五年":
    beyond_twoyr.append(1)
    below_twoyr.append(0)
  else:
    beyond_twoyr.append(np.nan)
    below_twoyr.append(np.nan)

In [231]:
prp = df["property_belong"]
prpp = np.array(prp)
for i in range(len(prpp)):
  prpp[i]=str(prpp[i])
print(np.unique(prpp))

['共有' '非共有']


In [232]:
shared = []
for i in range(len(prpp)):
  if prpp[i]=="共有":
    shared.append(1)
  else:
    shared.append(0)
df["shared"]=shared

C:\Users\Lenovo\AppData\Local\Temp/ipykernel_23440/2466874724.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df["shared"]=shared


In [233]:
upp = df["upload_photo"]
uppp = np.array(upp)
for i in range(len(uppp)):
  uppp[i]=str(uppp[i])
print(np.unique(uppp))

['已上传房本照片' '未上传房本照片']


In [234]:
uploaded = []
for i in range(len(uppp)):
  if uppp[i]=="已上传房本照片":
    uploaded.append(1)
  else:
    uploaded.append(0)
df["photo_uploaded"] = uploaded

C:\Users\Lenovo\AppData\Local\Temp/ipykernel_23440/1275203964.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df["photo_uploaded"] = uploaded


In [235]:
locc = df["loc"]
loccc = np.array(locc)
for i in range(len(loccc)):
  loccc[i]=str(loccc[i])
print(np.unique(loccc))

['' '\\xa0' '\\xa0中环至外环' '\\xa0内环内' '\\xa0内环至中环' '\\xa0外环外' '\\xa1内环内'
 '外环外']


In [236]:
central_out = []
inner = []
inner_central = []
for i in range(len(loccc)):
  if loccc[i]=="中环至外环":
    central_out.append(1)
    inner.append(0)
    inner_central.append(0)
  elif loccc[i]=="内环内":
    central_out.append(0)
    inner.append(1)
    inner_central.append(0)
  elif loccc[i]=="内环至中环":
    central_out.append(0)
    inner.append(0)
    inner_central.append(1)
  elif loccc[i]=="外环外":
    central_out.append(0)
    inner.append(0)
    inner_central.append(0)
  else:
    central_out.append(np.nan)
    inner.append(np.nan)
    inner_central.append(np.nan)
df["central_out"]=central_out
df["inner"]=inner
df["inner_central"]=inner_central

C:\Users\Lenovo\AppData\Local\Temp/ipykernel_23440/1510627760.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df["central_out"]=central_out
C:\Users\Lenovo\AppData\Local\Temp/ipykernel_23440/1510627760.py:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df["inner"]=inner
C:\Users\Lenovo\AppData\Local\Temp/ipykernel_23440/1510627760.py:27: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once u

In [237]:
avt = df["avail_visit_time"]
avtt = np.array(avt)
for i in range(len(avtt)):
  avtt[i]=str(avtt[i])
  tt = avtt[i]
  tt = tt.strip('"')
  tt = tt.strip(']')
  tt = tt.strip("'")
  tt = tt.strip("\\")
  df["avail_visit_time"][i]==tt
  avtt[i]=tt
print(np.unique(avtt))

['' '下班后可看' '具体信息请致电经纪人' '只周末可看' '提前预约随时可看' '有租户需预约']


In [238]:
after_work = []
call_agent = []
only_weekend = []
reserve_needed = []
for i in range(len(avtt)):
  if avtt[i]=="下班后可看":
    after_work.append(1)
    call_agent.append(0)
    only_weekend.append(0)
    reserve_needed.append(0)
  elif avtt[i]=="具体信息请致电经纪人":
    after_work.append(0)
    call_agent.append(1)
    only_weekend.append(0)
    reserve_needed.append(0)
  elif avtt[i]=="只周末可看":
    after_work.append(0)
    call_agent.append(0)
    only_weekend.append(1)
    reserve_needed.append(0)
  elif avtt[i]=="提前预约随时可看":
    after_work.append(0)
    call_agent.append(0)
    only_weekend.append(0)
    reserve_needed.append(1)
  elif avtt[i]=="有租户需预约":
    after_work.append(0)
    call_agent.append(0)
    only_weekend.append(0)
    reserve_needed.append(0)
  else:
    after_work.append(np.nan)
    call_agent.append(np.nan)
    only_weekend.append(np.nan)
    reserve_needed.append(np.nan)
df["after_work"] = after_work
df["call_agent"] = call_agent
df["only_weekend"]=only_weekend
df["reserve_needed"]=reserve_needed


C:\Users\Lenovo\AppData\Local\Temp/ipykernel_23440/3960756356.py:36: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df["after_work"] = after_work
C:\Users\Lenovo\AppData\Local\Temp/ipykernel_23440/3960756356.py:37: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df["call_agent"] = call_agent
C:\Users\Lenovo\AppData\Local\Temp/ipykernel_23440/3960756356.py:38: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all colum

In [ ]:
df.to_csv("2022928934.csv",index=False)

In [ ]:
dropped_col = ["rel_floor","config","layout","building_type","direction","structure","renovation_status","transact_property","function","age_limit","property_belong","upload_photo","loc","avail_visit_time"]
df = df.drop(dropped_col,axis=1)

In [ ]:
df = df.drop("no_of_units",axis=1)

In [ ]:
df.to_csv("2022928941dropped.csv",index=False)

In [ ]:
district = df["district"]
dist = np.array(district)
for i in range(len(dist)):
  dist[i]=str(dist[i])
print(np.unique(dist))

In [ ]:
df2 = pd.get_dummies(pd.Series(district))

In [ ]:
df_all = pd.concat([df,df2],axis=1)

In [ ]:
df_all = df_all.drop("district",axis=1)

In [ ]:
df_all.to_csv("C:/Users/Lenovo/Desktop/git/STAT4011_Project1/data/Lianjia.csv",index=False)

*east* column is deleted in outputed csv table.